In [1]:
import boxey as bx
import numpy as np
from numpy.typing import ArrayLike
from boxey import Process, Input, Model
import matplotlib.pyplot as plt
import yaml
import random
plt.rcParams.update({'font.size': 16})

In [2]:
from core import Problem, UniformBounded
from sampler import MCMCSampler

In [3]:
# List the names of the compartments
compartments = ['soil_prec', 'soil_pfsa', 'gw_prec', 'gw_pfsa']

def get_model(scenario, training_volume, fire_volume, R_soil_prec, R_soil_pfsa, R_gw_prec, R_gw_pfsa, k_bio):
    
    with open(f'data_and_constraints/{scenario}.yaml', 'r') as stream:
        data = yaml.safe_load(stream)

    k_soil = data['k_soil']
    k_gw = data['k_gw']
    
    c_prec = data['c_prec']
    c_pfsa = data['c_pfsa']

    # List the processes to represent.
    # Process needs: (name, timescale, compartment of origin, destination compartment)
    processes = [ Process('soil2gw_prec', R_soil_prec/k_soil, 'soil_prec', 'gw_prec'),
                Process('soil2gw_pfsa', R_soil_pfsa/k_soil, 'soil_pfsa', 'gw_pfsa'),
                Process('soil_prec2pfsa', 1/k_bio, 'soil_prec', 'soil_pfsa'),
                Process('gw_prec2pfsa', 1/k_bio, 'gw_prec', 'gw_pfsa'),
                Process('gw_prec', R_gw_prec/k_gw, 'gw_prec', None),
                Process('gw_pfsa', R_gw_pfsa/k_gw, 'gw_pfsa', None),
            
            ]

    # List the inputs to use. 
    # in the 0 inputs bookending these make it go from e.g. 0 at exactly 1970 to c_prec*V a tiny time later
    inputs = [ Input('AFFF_training_prec', [0., c_prec * training_volume, c_prec * training_volume, 0.], [1970.0, 1970.001, 1985.999, 1986.0], 'soil_prec'),
            Input('AFFF_training_pfsa', [0., c_pfsa * training_volume, c_pfsa * training_volume, 0.], [1970.0, 1970.001, 1985.999, 1986.0], 'soil_pfsa'),
            Input('AFFF_fire_prec', [0., c_prec * fire_volume, c_prec * fire_volume, 0.], [1997.0, 1997.001, 1997.999, 1998.0], 'soil_prec'),
            Input('AFFF_fire_pfsa', [0., c_pfsa * fire_volume, c_pfsa * fire_volume, 0.], [1997.0, 1997.001, 1997.999, 1998.0], 'soil_pfsa'),]

    model = bx.create_model(compartments, processes)
    model = bx.add_inputs(model, inputs)

    return model

In [4]:
def AFFF_use_history(scenario, yearly_training_volume, fire_volume, R_soil_prec, R_soil_pfsa, R_gw_prec, R_gw_pfsa, k_bio,
                    time_points=None):
    """To make iterating easier for optimizing."""
    model = get_model(scenario, training_volume = yearly_training_volume, fire_volume = fire_volume,
                      R_soil_prec = R_soil_prec,
                      R_soil_pfsa = R_soil_pfsa, 
                      R_gw_prec = R_gw_prec,
                      R_gw_pfsa = R_gw_pfsa,
                      k_bio = k_bio)
    if time_points is None:
        tstart, tend = 1970, 2100
        time_points = np.arange(tstart, tend, 1)
        time_points_in = sorted(time_points)
    else:
        time_points_in = sorted(np.unique([1970]+list(time_points)))
    reservoirs, times = model.run(time_points_in, initial_conditions=None)

    gw_pfsa_reservoir = np.array([reservoirs[times.index(t),3] for t in time_points[:-1]])
    gw_prec_reservoir = np.array([reservoirs[times.index(t),2] for t in time_points[:-1]])
    gw_pfsa_prec_ratio = np.mean([ratio for ratio in gw_pfsa_reservoir / gw_prec_reservoir if np.isfinite(ratio)])

    soil_pfsa_reservoir = reservoirs[-1, 1]
    
    return (gw_pfsa_reservoir, gw_pfsa_prec_ratio, soil_pfsa_reservoir)

def AFFF_use_history_log(scenario, yearly_training_volume, fire_volume, R_soil_prec, R_soil_pfsa, R_gw_prec, R_gw_pfsa, k_bio,
                    time_points=None):
    """To make iterating easier for optimizing."""
    return AFFF_use_history(scenario, 10**yearly_training_volume, 10**fire_volume, 10**R_soil_prec, 10**R_soil_pfsa,
                            10**R_gw_prec, 10**R_gw_pfsa, 10**k_bio,
                            time_points)

In [5]:
# class MyPrior(UniformBounded):
#     pass # just a bounded prior for now. Change as you wish

class MyPrior():
    
    def __init__(self, prior_type, param_1, param_2):
        self.prior_type = prior_type
        self.param_1 = param_1
        self.param_2 = param_2
    
    def uniform(self, proposal, idx):
        if self.param_1[idx] <= proposal <= self.param_2[idx]:
            return(0.)
        else:
            return(-1E6)
    
    def bounded_normal(self, proposal, idx):
        if proposal >= 0.:
            return(-0.5 * ((10**proposal - 10**self.param_1[idx]) / 10**self.param_2[idx])**2)
        else:
            return(-1E6)
        
    def __call__(self, proposal : ArrayLike) -> float:
        
        prior = [self.uniform(val, idx) if self.prior_type[idx] == 'U' else self.bounded_normal(val, idx) for idx, val in enumerate(proposal)]
        return(np.sum(prior))

class MyLikelihood:
    def __init__(self, scenario):
        self.scenario = scenario
        with open(f'data_and_constraints/{scenario}.yaml', 'r') as stream:
            data = yaml.safe_load(stream)

        gw_year = np.array(data['gw_year'])
        self.year = np.append(gw_year, data['soil_year'])
        self.gw_pfsa_reservoir = data['gw_pfsa_reservoir']
        self.log_soil_pfsa_reservoir = np.log10(data['soil_pfsa_reservoir'])
        self.gw_pfsa_prec_ratio = np.array(data['gw_pfsa_prec_ratio'])
        
    def __call__(self, params):
        modeled_gw_pfsa_reservoir, modeled_gw_pfsa_prec_ratio, modeled_soil_pfsa_reservoir = AFFF_use_history_log(
            self.scenario,
            params[0], params[1], params[2], params[3], params[4], params[5], params[6], time_points=self.year)
        
        #minimize the sum of squared errors for groundwater pfsa concentrations
        #minimize the sum of squared errors for groundwater pfsa concentrations
        likelihood = -np.sum((np.log10(self.gw_pfsa_reservoir) - np.log10(modeled_gw_pfsa_reservoir))**2)
        
        #penalize likelihood if soil pfsa reservoir proposal is more or less than one order of magnitude
        #from measurement
        if self.log_soil_pfsa_reservoir - 0.1 < np.log10(modeled_soil_pfsa_reservoir) < self.log_soil_pfsa_reservoir + 1:
            likelihood += 0
        else:
            likelihood += -1E6
            
        #penalize likelihood if gw pfsa to precursor ratio is more or less than two standard deviations
        #from measurements
        if self.gw_pfsa_prec_ratio[0] - 2 * self.gw_pfsa_prec_ratio[1] < modeled_gw_pfsa_prec_ratio < self.gw_pfsa_prec_ratio[0] + 2 * self.gw_pfsa_prec_ratio[1]:
            likelihood += 0
        else:
        
            likelihood += -1E6
        
        #enforce Rrec > Rpfaa for given compartment
        if params[2] > params[3]:
            likelihood += 0
        else:
            likelihood += -1E6
        
        if params[4] > params[5]:
            likelihood += 0
        else:
            likelihood += -1E6
            
        return(likelihood)

class MyProblem(Problem):

    def __init__(self, n_dimensions, lower_bounds, upper_bounds, scenario):
        self.n_dimensions = n_dimensions
        self.lower_bounds = np.array(lower_bounds)
        self.upper_bounds = np.array(upper_bounds)
        self.prior = MyPrior(prior_type = ['U', 'U', 'U', 'U', 'U', 'U', 'U'],
            param_1=self.lower_bounds, param_2=self.upper_bounds)
        self.likelihood = MyLikelihood(scenario)

    def get_bounds(self):
        return self.lower_bounds, self.upper_bounds

In [6]:
def fta_pfc(scenario):
    with open(f'data_and_constraints/{scenario}.yaml', 'r') as stream:
        data = yaml.safe_load(stream)
        
    yearly_training_volume = np.log10(np.array(data['V_training']))
    fire_volume = np.log10(np.array(data['V_fire']))
    R_soil_prec = np.log10(np.array(data['R_soil_prec']))
    R_soil_pfsa = np.log10(np.array(data['R_soil_pfsa']))
    R_gw_prec = np.log10(np.array(data['R_gw_prec']))
    R_gw_pfsa = np.log10(np.array(data['R_gw_pfsa']))
    k_bio = np.log10(np.array(data['k_bio']))
    
    problem = MyProblem(7,
        [yearly_training_volume[0], fire_volume[0], R_soil_prec[0], R_soil_pfsa[0], R_gw_prec[0], R_gw_pfsa[0], k_bio[0]],
        [yearly_training_volume[1], fire_volume[1], R_soil_prec[1], R_soil_pfsa[1], R_gw_prec[1], R_gw_pfsa[1], k_bio[1]],
        scenario = scenario)

    return(problem)

In [7]:
sampler = MCMCSampler(max_steps=100000, Nwalkers=4, Nincrement=500, target_effective_steps=2500)

In [8]:
C8_pfsa = fta_pfc('C8_pfsa')
C8_pfsa_posterior = sampler.sample(problem = C8_pfsa, alpha = 0.3)
C8_pfsa_posterior_samples = 10**C8_pfsa_posterior.samples
np.savetxt('posterior_samples/C8_pfsa_posterior.csv', C8_pfsa_posterior_samples, delimiter = ',')

/var/folders/vf/0txmsx4s6yg188n7v90wwnfm0000gn/T/ipykernel_27393/1226582035.py:20: RuntimeWarning: divide by zero encountered in true_divide
  gw_pfsa_prec_ratio = np.mean([ratio for ratio in gw_pfsa_reservoir / gw_prec_reservoir if np.isfinite(ratio)])


acceptance rate is 0.58 when alpha is 0.3
Sampling posterior in 500-iteration increments.
After 500 iterations, autocorr time: unavailable
After 1000 iterations, autocorr time: unavailable
After 1500 iterations, autocorr time: unavailable
After 2000 iterations, autocorr time: unavailable
After 2500 iterations, autocorr time: unavailable
After 3000 iterations, autocorr time: unavailable
After 3500 iterations, autocorr time: unavailable
After 4000 iterations, autocorr time: unavailable
After 4500 iterations, autocorr time: unavailable
After 5000 iterations, autocorr time: unavailable
After 5500 iterations, autocorr time: unavailable
After 6000 iterations, autocorr time: unavailable
After 6500 iterations, autocorr time: unavailable
After 7000 iterations, autocorr time: unavailable
After 7500 iterations, autocorr time: unavailable
After 8000 iterations, autocorr time: unavailable


/var/folders/vf/0txmsx4s6yg188n7v90wwnfm0000gn/T/ipykernel_27393/1226582035.py:20: RuntimeWarning: overflow encountered in true_divide
  gw_pfsa_prec_ratio = np.mean([ratio for ratio in gw_pfsa_reservoir / gw_prec_reservoir if np.isfinite(ratio)])


After 8500 iterations, autocorr time: unavailable
After 9000 iterations, autocorr time: unavailable
After 9500 iterations, autocorr time: unavailable
After 10000 iterations, autocorr time: unavailable
After 10500 iterations, autocorr time: unavailable
After 11000 iterations, autocorr time: 219.65043374715268
After 11500 iterations, effective number of samples:                    1494
After 12000 iterations, effective number of samples:                    1569
After 12500 iterations, effective number of samples:                    1655
After 13000 iterations, effective number of samples:                    1721
After 13500 iterations, effective number of samples:                    1781
After 14000 iterations, effective number of samples:                    1865
After 14500 iterations, effective number of samples:                    1925
After 15000 iterations, effective number of samples:                    2005
After 15500 iterations, effective number of samples:                    20

In [ ]:
C6_pfsa = fta_pfc('C6_pfsa')
C6_pfsa_posterior = sampler.sample(problem = C6_pfsa, alpha = 0.4)
C6_pfsa_posterior_samples = 10**C6_pfsa_posterior.samples
np.savetxt('posterior_samples/C6_pfsa_posterior.csv', C6_pfsa_posterior_samples, delimiter = ',')

In [ ]:
C4_pfsa = fta_pfc('C4_pfsa')
C4_pfsa_posterior = sampler.sample(problem = C4_pfsa, alpha = 0.4)
C4_pfsa_posterior_samples = 10**C4_pfsa_posterior.samples
np.savetxt('posterior_samples/C4_pfsa_posterior.csv', C4_pfsa_posterior_samples, delimiter = ',')